In [1]:
from contextlib import contextmanager
import sqlite3
import yaml

In [2]:
curr_dir = ! pwd
curr_dir = curr_dir[0]
print(curr_dir)
assert curr_dir.endswith("hyperborea3/scripts/spells")

/home/jeremy/projects/hyperborea3/scripts/spells


In [3]:
load_folder = "lvl6"

In [4]:
@contextmanager
def db_cur():
    URI = f"../../hyperborea3/hyperborea.sqlite3"
    con = sqlite3.connect(URI, check_same_thread=False)
    con.row_factory = sqlite3.Row
    cur = con.cursor()
    yield cur
    con.commit()
    con.close()

Test connection to db

In [5]:
with db_cur() as cur:
    cur.execute("""
        SELECT *
          FROM spells;
    """)
    result = [dict(x) for x in cur.fetchall()]
result[:1]

[{'spell_id': 1,
  'spell_name': 'Acid Arrow',
  'rng': '30 feet',
  'dur': 'special',
  'reversible': 0,
  'pp': 175,
  'spell_desc': '<p>A magic arrow darts from the finger of the caster. On a successful attack roll (<i>dexterity</i> modifier applies), the <i>acid arrow</i> strikes for 1d4+1 hp physical damage, plus an additional 2d4 hp acid damage in the same round. Magicians (but not other sorcerers) enjoy a +1 bonus to the attack roll for every 2 CA levels (CA 3–4 = +2, CA 5–6 = +3, and so forth). Acid damage will persist for higher level sorcerers:</p> <ul><li>1 extra round for CA 4–6</li> <li>2 extra rounds for CA 7–9</li> <li>3 extra rounds for CA 10 or greater.</li></ul> <p>For example, an <i>acid arrow</i> fired by a CA 12 sorcerer on round 1 would inflict 1d4+1 hp base damage plus 2d4 hp acid damage on round 1, 2d4 hp acid damage on round 2, 2d4 hp acid damage on round 3, and a final 2d4 hp acid damage on round 4. The acid may ruin armour or clothing per referee discretion. 

In [6]:
files = ! ls {load_folder}/*.yml
files

['lvl6/Acid_Fog.yml',
 'lvl6/Animate_Dead_II.yml',
 'lvl6/Animate_Objects.yml',
 'lvl6/Anti-Beast_Shell.yml',
 'lvl6/Blade_Barrier.yml',
 'lvl6/Chain_Lightning.yml',
 'lvl6/Chill_of_Death.yml',
 'lvl6/Communicate_with_Monsters.yml',
 'lvl6/Control_Lycanthrope.yml',
 'lvl6/Control_Weather.yml',
 'lvl6/Death_Ray_of_Immolation.yml',
 'lvl6/Demi-Shadow_Sorcery.yml',
 'lvl6/Disintegrate.yml',
 'lvl6/Extend_Spell_III.yml',
 'lvl6/Find_the_Path.yml',
 'lvl6/Fire_Seeds.yml',
 'lvl6/Forbiddance.yml',
 'lvl6/Forceful_Hand.yml',
 'lvl6/Freezing_Sphere.yml',
 'lvl6/Geas.yml',
 'lvl6/Globe_of_Invulnerability,_Greater.yml',
 'lvl6/Guards_and_Wards.yml',
 'lvl6/Heal.yml',
 'lvl6/Heroes’_Feast.yml',
 'lvl6/Iceberg.yml',
 'lvl6/Legend_Lore.yml',
 'lvl6/Liveoak.yml',
 'lvl6/Mass_Suggestion.yml',
 'lvl6/Mirage_Arcana.yml',
 'lvl6/Mislead.yml',
 'lvl6/Move_Earth.yml',
 'lvl6/Permanent_Phantasm.yml',
 'lvl6/Phantasmagorical_Precipitousness.yml',
 'lvl6/Prismatic_Sphere.yml',
 'lvl6/Programmed_Phantasm.yml'

In [7]:
def load_yml_spell(file_name):
    with open(file_name, "r") as f:
        spell = yaml.safe_load(f)
    return spell

In [14]:
def spell_preload_check(spell):
    with db_cur() as cur:
        cur.execute(
            """
            SELECT *
            FROM spells
            WHERE spell_id = ?;
            """,
            (spell["id"],),
        )
        db_spell = cur.fetchone()
    assert spell["name"] == db_spell["spell_name"]
    assert isinstance(spell["id"], int)
    assert isinstance(spell["pp"], int)
    assert isinstance(spell["Rng"], (str, int))
    assert isinstance(spell["Dur"], str)
    assert spell["Rev"] in [0, 1]
    assert isinstance(spell["Desc"], str)
    assert spell["Desc"][:3] == "<p>"
    if spell["id"] not in [53, 75, 105, 346]:
        assert spell["Desc"][-4:] == "</p>"



In [15]:
def update_spell(spell):
    with db_cur() as cur:
        cur.execute(
            """
            UPDATE spells
            SET rng = ?
                , dur = ?
                , reversible = ?
                , pp = ?
                , spell_desc = ?
            WHERE spell_id = ?;
            """,
            (
                spell["Rng"],
                spell["Dur"],
                spell["Rev"],
                spell["pp"],
                spell["Desc"],
                spell["id"],
            ),
        )
    return 

### Test parsing all the files

In [16]:
for f in files:
    print(f"checking {f}")
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    print("success")
    print()

checking lvl6/Acid_Fog.yml
success

checking lvl6/Animate_Dead_II.yml
success

checking lvl6/Animate_Objects.yml
success

checking lvl6/Anti-Beast_Shell.yml
success

checking lvl6/Blade_Barrier.yml
success

checking lvl6/Chain_Lightning.yml
success

checking lvl6/Chill_of_Death.yml
success

checking lvl6/Communicate_with_Monsters.yml
success

checking lvl6/Control_Lycanthrope.yml
success

checking lvl6/Control_Weather.yml
success

checking lvl6/Death_Ray_of_Immolation.yml
success

checking lvl6/Demi-Shadow_Sorcery.yml
success

checking lvl6/Disintegrate.yml
success

checking lvl6/Extend_Spell_III.yml
success

checking lvl6/Find_the_Path.yml
success

checking lvl6/Fire_Seeds.yml
success

checking lvl6/Forbiddance.yml
success

checking lvl6/Forceful_Hand.yml
success

checking lvl6/Freezing_Sphere.yml
success

checking lvl6/Geas.yml
success

checking lvl6/Globe_of_Invulnerability,_Greater.yml
success

checking lvl6/Guards_and_Wards.yml
success

checking lvl6/Heal.yml
success

checking lvl

Do it for real

In [17]:
for f in files:
    spell = load_yml_spell(f)
    spell_preload_check(spell)
    update_spell(spell)

In [18]:
load_yml_spell(files[-1])

{'id': 451,
 'name': 'Word of Recall',
 'pp': 246,
 'Lvl': ['clr 6'],
 'Rng': 0,
 'Dur': 'instantaneous',
 'Rev': 0,
 'Desc': '<p>The chanting sorcerer transports to a place that they hallowed with the <i>ceremony of consecration</i> spell, with no chance of error; no distance or subterranean depth is too great. The sorcerer may bring 25 pounds of material per CA level; this excess may comprise gear, treasure, or even a living creature, so long as the weight restriction is observed.</p>'}